In [13]:
# initialize sql context
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

In [14]:
#Todo: using SQLContext to read csv and assign to dataframe
df = sqlContext.read.csv('abfss://synapsedatalake@synapseaiadadls.dfs.core.windows.net/pocdata.csv', header=True, inferSchema= True)	

In [15]:
#Todo:printSchema
df.printSchema()

In [16]:
# Run the cell to rename the columns properly:
cols = ['CUSTID','Gender','Age_Band','State','LGA','Account_Open_Date','Tenure','Occupation','Education','Segment','Marital','No_of_Accounts','GDP_in_Billions_of_USSD','Inflation','Population','txn_amount_M1','txn_vol_M1','txn_amount_M2','txn_vol_M2','txn_amount_M3','txn_vol_M3','F1','F2','Latest_TxnDate','Recency','Freq_M1','Freq_M2','F3','CHURN']

#note income -renamed-> as label
df=df.toDF(*cols)

         

In [17]:
# Import all from `sql.types`
from pyspark.sql.types import *

# Write a custom function to convert the data type of DataFrame columns
def convertColumn(df, names, newType):
    for name in names: 
        df = df.withColumn(name, df[name].cast(newType))
    return df 
# List of continuous features
CONTI_FEATURES  = ['GDP_in_Billions_of_USSD','Inflation','Population','txn_amount_M1','txn_vol_M1','txn_amount_M2','txn_vol_M2','txn_amount_M3','txn_vol_M3','F1']
# Convert the type
df = convertColumn(df, CONTI_FEATURES, FloatType())
# Check the dataset
df.printSchema()

In [18]:
df.show(5, truncate = False)

In [19]:
#todo: describe data,column by count,by group
df.describe().show()

In [20]:
# the summary statistic of only one column, add the name of the column inside describe()

df.describe('txn_vol_m1').show()

In [21]:
#Descriptive statistics by group
df.groupBy("Marital").count().sort("count",ascending=True).show()

In [22]:
#example
df.groupby('Age_Band').agg({'txn_vol_m2': 'mean'}).show()		


In [23]:
#todo crosstab computation
df.crosstab('Age_Band', 'CHURN').sort("Age_Band_CHURN").show()

In [24]:
#Drop null vals
#df  = df.drop('Population', 'Inflation','State','LGA')

In [25]:
#TODO:  # Add age square
        
#from pyspark.sql.functions import *

# 1 Select the column
#age_square = df.select(col("age")**2)

# 2 Apply the transformation and add it to the DataFrame
#df = df.withColumn("age_square", col("age")**2)

df.printSchema()

In [26]:
#TODO : follow the above instruction
#df.filter(df.Segment == 'HNI').count()
#df.groupby('Segment').agg({'Segment': 'count'}).sort(asc("count(Segment)")).show()

In [27]:
df_remove = df.filter(df.Recency!='B')

In [28]:
#import libraries for pipeline
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

In [29]:
# 1. Encode the categorical data
CATE_FEATURES = ['Gender','Education', 'Marital', 'Segment', 'Occupation', 'F2', 'Recency', 'Freq_M1','Freq_M2','F3']

# stages in our Pipeline
stages = [] 


for categoricalCol in CATE_FEATURES:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()],
                                     outputCols=[categoricalCol + "classVec"])
    
    stages += [stringIndexer, encoder]

In [30]:
# 2. Index the label feature
# Convert label into label indices using the StringIndexer
label_stringIdx =  StringIndexer(inputCol="CHURN", outputCol="newCHURN")
stages += [label_stringIdx]

In [31]:
# 3. Add continuous variable
assemblerInputs = [c + "classVec" for c in CATE_FEATURES] + CONTI_FEATURES

In [32]:
# 4. Assemble the steps
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [33]:
stages

In [34]:
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(df_remove)
model = pipelineModel.transform(df_remove)

In [35]:
model

In [36]:
# To make the computation faster, you convert model to a DataFrame.
#You need to select newlabel and features from model using map.

from pyspark.ml.linalg import DenseVector
input_data = model.rdd.map(lambda x: (x["newCHURN"], DenseVector(x["features"])))

In [37]:
# import 
# from pyspark.ml.linalg import DenseVector
df_train = sqlContext.createDataFrame(input_data, ["CHURN", "features"])

In [38]:
df_train.show(2)

In [39]:
# Split the data into train and test sets
train_data, test_data = df_train.randomSplit([.8,.2],seed=1234)

In [40]:
# Let's count how many people with income below/above 50k in both training and test set
train_data.groupby('CHURN').agg({'CHURN': 'count'}).show()	

In [41]:
test_data.groupby('CHURN').agg({'CHURN': 'count'}).show()

In [42]:
#You initialize lr by indicating the label column and feature columns. 
# Import `LogisticRegression`
from pyspark.ml.classification import LogisticRegression

# Initialize `lr`
lr = LogisticRegression(labelCol="CHURN",
                        featuresCol="features",
                        maxIter=10,
                        regParam=0.3)

# Fit the data to the model
linearModel = lr.fit(train_data)

In [43]:
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(linearModel.coefficients))
print("Intercept: " + str(linearModel.intercept))

In [44]:
#To generate prediction for your test set, you can use linearModel with transform() on test_data
# Make predictions on test data using the transform() method.
predictions = linearModel.transform(test_data)

In [45]:
predictions.printSchema()

In [46]:
selected = predictions.select("CHURN", "prediction", "probability")
selected.show(20)

In [47]:
#We need to look at the accuracy metric to see how well (or bad) the model performs.

def accuracy_m(model): 
    predictions = model.transform(test_data)
    cm = predictions.select("CHURN", "prediction")
    acc = cm.filter(cm.CHURN == cm.prediction).count() / cm.count()
    print("Model accuracy: %.3f%%" % (acc * 100)) 

accuracy_m(model = linearModel)